In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Code by Andrey Shtrauss https://www.kaggle.com/shtrausslearning/airfoil-noise-prediction-modeling-using-gpr

In [ ]:
''' Import Modules '''

import sys
import warnings

from pandas import set_option
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
        
if not sys.warnoptions:
    warnings.simplefilter('ignore')

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadsgermancsv/german.csv', delimiter=';', encoding = "ISO-8859-2", nrows = nRowsRead)
df.dataframeName = 'german.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head().style.set_properties(**{'background-color':'LightBlue',
                                     'color': 'purple'})

In [ ]:
!pip install seaborn --upgrade

In [ ]:
import seaborn as sns
print(sns.__version__)

In [ ]:
# Plot Correlation to Target Variable only
def corrMat(df,target='Creditability',figsize=(9,0.5),ret_id=False):
    
    corr_mat = df.corr().round(2);shape = corr_mat.shape[0]
    corr_mat = corr_mat.transpose()
    corr = corr_mat.loc[:, df.columns == target].transpose().copy() 
    
    if(ret_id is False):
        f, ax = plt.subplots(figsize=figsize)
        sns.heatmap(corr,vmin=-0.3,vmax=0.3,center=0, 
                     cmap='viridis',square=False,lw=2,annot=True,cbar=False)
        plt.title(f'Feature Correlation to {target}')
    
    if(ret_id):
        return corr

In [ ]:
corrMat(df,'Duration_of_Credit_monthly',figsize=(7,0.5))

#EDA: PairGrids

In [ ]:
lst_vars = df.columns.to_list();lst_vars

#I grouped since the charts were very messy with all data. Now, the charts are clear with less features.

In [ ]:
df_grp = df.groupby(["Creditability","Duration_of_Credit_monthly"])[["Credit_Amount","Account_Balance","Age_years"]].sum().reset_index()
df_grp.head()

In [ ]:
''' Draw a Bivariate Seaborn Pairgrid /w KDE density w/ '''
def snsPairGrid(df):

    sns.set(style='whitegrid')
    g = sns.PairGrid(df,diag_sharey=False,height=4)
    g.fig.set_size_inches(15,15)
    g.map_diag(sns.kdeplot, lw=2)
    g.map_lower(sns.scatterplot,s=25,edgecolor="k",linewidth=0.5,alpha=0.4)
    g.map_lower(sns.kdeplot,cmap='plasma',n_levels=6,alpha=0.5)
    plt.tight_layout()

In [ ]:
snsPairGrid(df_grp)

#Frequency Domain Creditability Data Analysis

In [ ]:
def plotRel(ldf):
    sns.set(style='whitegrid')
    g = sns.relplot(x='Duration_of_Credit_monthly',y='Creditability',col='Credit_Amount',row='Account_Balance',data=ldf,
                kind='line',legend='full',hue='Age_years',palette='jet_r',height=4)

In [ ]:
''' Create Data Subsets for different airfoil sizes '''
sns.set(style='whitegrid')
pdlst_creditability = [] # list of pd subset data storing all unique 'l_chord'
lst_creditability_unique = df_grp['Creditability'].unique().tolist();lst_creditability_unique.sort() # get unique 'l_chord'&sort()
for i in lst_creditability_unique:
    ldf = df_grp[df_grp['Creditability']==i].copy()
    pdlst_creditability.append(ldf)    # add all unique subsets to list

In [ ]:
# lets plot two cases, they are ordered so smaller one on top.
plotRel(pdlst_creditability[0])
plotRel(pdlst_creditability[1])

#I didn't expect for those "charts" above. The original was 

plotRel(pdlst_creditability[0])

plotRel(pdlst_creditability[2])

In [ ]:
sns.set(style='whitegrid')
lst = [0,2]
def relplot1(Creditability):
    for i in Creditability:
        g = sns.relplot(x='Duration_of_Credit_monthly',y='Creditability',col='Credit_Amount',row='Creditability',
                    hue='Age_years',sizes='Account_Balance',palette='jet_r',
                    kind='scatter',legend='full',data=pdlst_creditability[1])
        g.fig.set_size_inches(13,3)
        leg = g._legend
        leg.set_bbox_to_anchor([1.06,1])  # coordinates of lower left of bounding box
        leg._loc = 1  # if required you can set the loc

#I changed (i to 1) in data=pdlst_creditability[i]) to avoid error "list out of index". Though the messy "chart" below.

In [ ]:
relplot1(lst)
#plt.xticks(rotation=45)

#I gave up since the charts are not what I expected.